In [16]:
import os
from flask import Flask, request, render_template
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.inception_v3 import preprocess_input

# Initialize Flask app
app = Flask(__name__)

# Load the trained discriminator model
model_path = 'FINAL_discriminator_model.h5'
if not os.path.exists(model_path):
    print(f"Error: Model file '{model_path}' not found.")
    exit(1)

try:
    discriminator_model = load_model(model_path)
except Exception as e:
    print(f"Error loading model: {str(e)}")
    exit(1)

# Define a function to classify image
def classify_image(image_path, model):
    try:
        # Load and preprocess the input image
        img = load_img(image_path, target_size=(64, 64))
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)

        # Predict using the discriminator model
        prediction = model.predict(img_array)

        # Interpret the prediction result
        if prediction[0][0] > 0.5:
            result = "REAL"
        else:
            result = "FAKE"

        return result
    except Exception as e:
        return str(e)

# Define route for the homepage
@app.route('/')
def home():
    return render_template('index.html')

# Define route to handle image classification
@app.route('/classify', methods=['POST'])
def classify():
    try:
        if request.method == 'POST':
            # Get the file from the POST request
            file = request.files['file']

            if file.filename == '':
                return render_template('result.html', result="No file selected.")

            # Save the file to the uploads folder
            file_path = os.path.join('uploads', file.filename)
            file.save(file_path)

            # Classify the uploaded image
            classification_result = classify_image(file_path, discriminator_model)

            return render_template('result.html', result=classification_result)
    except Exception as e:
        return render_template('result.html', result=str(e))

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Mar/2024 16:25:59] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


INFO:werkzeug:127.0.0.1 - - [21/Mar/2024 16:26:03] "POST /classify HTTP/1.1" 200 -
